### input : 
    Fact checking phrase on recent news about politics
### output : 
    Goes to next function that queries alchemy news and databases
#### types of facts: 
    person facts : list of person names
    country facts : list of countries involved
    organization facts : list of org (UN, etc.)
    
#### verbs and relations : 
    if person: resides in, works for, employed by, 
    countries / organizations : affected by, agent of 
    
### TODO :
    Retrieve more entities than AlchemyAPI gives

In [11]:
import json
from API_Key import *
from watson_developer_cloud import AlchemyLanguageV1

alchemy_language = AlchemyLanguageV1(api_key=api_key_yj)

from ipywidgets import widgets, interact
from IPython.display import display

In [2]:
def extract_keywords(inputfact):
    subjects={} #dictionary of people/organization/country
    subjects['Person']=[]
    subjects['Country']=[]
    subjects['Organization']=[]
    subjects['Unsorted Subjects']=[]
        
    relations=[] #list of relations/verbs
    objects=[]
        
    import re
    n_list=[]
    #negation
    negation=False
    for n in ["no", "not", "never", "none", "noone"] : 
        if n in re.compile('\w+').findall(inputfact):
            negation=True
            n_list.append(n)

    
    #keywords
    response = alchemy_language.combined(text=inputfact,extract='entities,typed-rels',max_items=100)
    if response['status'] == 'OK' : 
        
        
        for entity in response['entities']:
            e_type=entity['type']
            if e_type in subjects : #either person, country or organization
                subjects[e_type].append(entity['text'])
            elif (e_type=='JobTitle'):
                relations.append('worksAs')
                objects.append(entity['text'])

        for relation in response['typedRelations']:
            for argument in relation['arguments']:
                if argument['part']=='second' :
                    for entity in argument['entities']:
                        subjects['Unsorted Subjects'].append(entity['text'])
                if argument['part']=='second' :
                    for entity in argument['entities']:
                        objects.append(entity['text'])
            relations.append(relation['type'])
        
        print('This fact is about: ')
        for key in subjects.keys() :
            if not (len(subjects[key]) == 0) :
                print(key+" : ")
                print(subjects[key])
        print("")
        
        print("The relations and verbs involved are: ")
        print(relations)
        print("")
        
        print("The objects of these relations are: ")
        print(objects)
        print("")
    else:
        print('Error in keyword extaction call: ', response['statusInfo'])
    
    if (negation) : 
        print('')
        print('## Negations ##')
        if (len(n_list)==1) : 
            print("There is a negation : ", end='')
            print(n_list)
        else : 
            print("There are negations : ", end='')
            print(n_list)


    # Prepositions ? 

    # Semantic query expansion
    # dbpedia : Quepy
    # Scoping problem - Political issues. Sample queries
    print('')
    print('')
    
    output = {'subjects' : subjects, 
              'relations' : relations, 
              'objects' : objects, 
              'negations' : n_list}
    return output

In [ ]:
def search_sources(keywords) : 
    references = []
    words_found = []
    print ("Sources used : ")
    for ref in references : 
        print (ref)
    return words_found

In [ ]:
def compare(words_found, keywords):
    result = True
    print ("This fact is : "+str(result)+" with "+str(confidence)+" confidence.")

In [ ]:
while True: 
    print ("Enter a fact:")
    inputfact = input()
    keywords = extract_keywords(inputfact)
    
    # Lee Hsien Loong is the prime minister of Singapore
    # The UN president is Ban Ki Moon
    # The US is at war with Syria
    # Laos became a member of ASEAN in 2016
    
    # Pass keywords to search engine
    # search_sources prints sources found, returns a list of words to compare and % "trustability"
    words_found = search_sources(keywords)
    
    # Compare references and keywords
    # compare prints a true or false, with rate of confidence
    
    compare(words_found, keywords)

Enter a fact:
Laos became a member of ASEAN in 2016
This fact is about: 
Country : 
['Laos']
Organization : 
['ASEAN']
Unsorted : 
['ASEAN']

The relations and verbs involved are: 
['employedBy']

The objects of these relations are: 
['ASEAN']



Enter a fact:
Obama is the prime minister of UK
This fact is about: 
Country : 
['UK']
Person : 
['Obama']
Unsorted : 
['UK']

The relations and verbs involved are: 
['worksAs', 'residesIn']

The objects of these relations are: 
['prime minister', 'UK']



Enter a fact:
